In [1]:
from Modules import Ingenierie, data, simple_visualisation, advanced_visualisation, widget,data_cleaning
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.calibration import CalibrationDisplay, calibration_curve
import requests
import json
import math

# Chargement des données et création du dataframe

In [6]:
year_list = range(2016,2024)
#all_data=data.load_data(year_list)
dfs = data_cleaning.create_dataframes(year_list, data.load_data(year_list))

Loading data for season 2016
Data for season 2016 loaded successfully.
Loading data for season 2017
Data for season 2017 loaded successfully.
Loading data for season 2018
Data for season 2018 loaded successfully.
Loading data for season 2019
Data for season 2019 loaded successfully.
Loading data for season 2020
Data for season 2020 loaded successfully.
Loading data for season 2021
Data for season 2021 loaded successfully.
Loading data for season 2022
Data for season 2022 loaded successfully.
Loading data for season 2023
Data for season 2023 loaded successfully.
Sauvegardé: dataframe\season_2016_2016020001.csv
Sauvegardé: dataframe\season_2016_2016020002.csv
Sauvegardé: dataframe\season_2016_2016020003.csv
Sauvegardé: dataframe\season_2016_2016020004.csv
Sauvegardé: dataframe\season_2016_2016020005.csv
Sauvegardé: dataframe\season_2016_2016020006.csv
Sauvegardé: dataframe\season_2016_2016020007.csv
Sauvegardé: dataframe\season_2016_2016020008.csv
Sauvegardé: dataframe\season_2016_201602

In [ ]:
dataframes = data_cleaning.load_dataframes(year_list)
dataframes[2020].head(10)

,ID,Sort Order,Time,Period,Event ID,Team,Type,Coordinates,Shooter ID,Goalie ID,Shot Type,Empty Net,Force Type,Home
0,2020020001,10,00:16,1,53,Flyers,shot-on-goal,"(-74, 29)",Travis,Tristan,wrist,False,Even Strength,True
1,2020020001,13,00:34,1,9,Penguins,shot-on-goal,"(49, -25)",Evan,Carter,wrist,False,Even Strength,False
2,2020020001,19,01:05,1,55,Flyers,shot-on-goal,"(-52, -31)",Joel,Tristan,backhand,False,Even Strength,True
3,2020020001,39,02:51,1,62,Penguins,shot-on-goal,"(43, 39)",Evan,Carter,snap,False,Even Strength,False
4,2020020001,46,03:44,1,65,Flyers,shot-on-goal,"(-53, 8)",Claude,Tristan,slap,False,Even Strength,True
5,2020020001,47,03:46,1,66,Flyers,shot-on-goal,"(-70, -3)",Philippe,Tristan,wrist,False,Even Strength,True
6,2020020001,53,04:30,1,67,Penguins,shot-on-goal,"(59, -35)",Sam,Carter,wrist,False,Even Strength,False
7,2020020001,55,04:39,1,68,Flyers,shot-on-goal,"(-59, -28)",Travis,Tristan,wrist,False,Even Strength,True
8,2020020001,61,05:12,1,70,Penguins,goal,"(81, -5)",Mark,Carter,snap,False,Even Strength,False
9,2020020001,65,05:32,1,257,Penguins,shot-on-goal,"(32, 0)",Mike,Carter,wrist,False,Even Strength,False


In [17]:
df = dataframes[2020]
df.head(10)
def time_to_seconds(time_str):
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds

# Conversion du temps en secondes
df['Game seconds'] = df['Time'].apply(time_to_seconds)

# Renommer la colonne `Period` en `Game period`
df['Game period'] = df['Period']

# Séparation des coordonnées en colonnes `x_coord` et `y_coord`
df[['x_coord', 'y_coord']] = df['Coordinates'].str.extract(r'\((-?\d+),\s*(-?\d+)\)').astype(float)

# Calculer la distance de tir
def calculate_shot_distance(coord_str):
    x, y = map(int, coord_str.strip('()').split(','))
    goal_x = 89 if x > 0 else -89
    return math.sqrt((x - goal_x) ** 2 + y ** 2)

df['Shot distance'] = df['Coordinates'].apply(lambda coord: calculate_shot_distance(coord))

# Calculer l'angle de tir
def calculate_shot_angle(coord_str):
    x, y = map(int, coord_str.strip('()').split(','))
    goal_x = 89 if x > 0 else -89
    angle_radians = math.atan2(y, (goal_x - x))
    return math.degrees(angle_radians)

df['Shot angle'] = df['Coordinates'].apply(lambda coord: calculate_shot_angle(coord))

# Renommer `Shot Type` en `Shot type`
df['Shot type'] = df['Shot Type']

# 1. Dernier type d'événement (Last event type)
df['Last event type'] = df['Type'].shift(1)

# 2. Coordonnées du dernier événement
df['Last x_coord'] = df['x_coord'].shift(1)
df['Last y_coord'] = df['y_coord'].shift(1)

# 3. Temps écoulé depuis le dernier événement
df['Time since last event'] = df['Game seconds'] - df['Game seconds'].shift(1)

# 4. Distance depuis le dernier événement
def calculate_distance(x1, y1, x2, y2):
    if any(pd.isna([x1, y1, x2, y2])):  # Si une des coordonnées est NaN, retourner NaN
        return np.nan
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

df['Distance from last event'] = df.apply(
    lambda row: calculate_distance(row['x_coord'], row['y_coord'], row['Last x_coord'], row['Last y_coord']), axis=1
)

# Créer le DataFrame final avec toutes les colonnes nécessaires
df_final = df[[
    'Game seconds', 'Game period', 'x_coord', 'y_coord',
    'Shot distance', 'Shot angle', 'Shot type',
    'Last event type', 'Last x_coord', 'Last y_coord',
    'Time since last event', 'Distance from last event'
]]

# Afficher le DataFrame final
print(df_final)

       Game seconds  Game period  x_coord  y_coord  Shot distance  Shot angle  \
0                16            1    -74.0     29.0      32.649655  117.349876   
1                34            1     49.0    -25.0      47.169906  -32.005383   
2                65            1    -52.0    -31.0      48.270074 -140.042451   
3               171            1     43.0     39.0      60.307545   40.292148   
4               224            1    -53.0      8.0      36.878178  167.471192   
...             ...          ...      ...      ...            ...         ...   
57729           979            3     57.0     30.0      43.863424   43.152390   
57730          1033            3     56.0    -31.0      45.276926  -43.210089   
57731          1141            3    -45.0    -14.0      46.173586 -162.349876   
57732          1150            3    -77.0     -3.0      12.369317 -165.963757   
57733          1164            3    -77.0     18.0      21.633308  123.690068   

      Shot type Last event 

In [24]:
import pandas as pd
import math

def add_advanced_features(df):
    """
    Ajoute des caractéristiques avancées au DataFrame, incluant les rebonds, le changement d'angle,
    et la vitesse. Prend également en compte le changement de période.
    
    :param df: DataFrame contenant les informations des tirs.
    :return: DataFrame enrichi avec les nouvelles caractéristiques.
    """
    # Ajouter les colonnes des événements précédents (décalage de 1 ligne)
    df['prev_event_type'] = df['Type'].shift(1)
    df['prev_x_coord'] = df['x_coord'].shift(1)
    df['prev_y_coord'] = df['y_coord'].shift(1)
    df['prev_shot_angle'] = df['Shot angle'].shift(1)
    df['prev_game_seconds'] = df['Game seconds'].shift(1)
    df['prev_period'] = df['Period'].shift(1)  # Ajout de la période précédente

    # Rebond : Vérifie si le dernier événement était aussi un tir
    shot_types = ["shot-on-goal", "goal", "blocked-shot", "missed-shot"]
    df['Rebound'] = df['prev_event_type'].apply(lambda x: x in shot_types)

    # Changement d'angle de tir : Différence d'angle si c'est un rebond
    def calculate_angle_change(row):
        if row['Rebound']:
            return abs(row['Shot angle'] - row['prev_shot_angle'])
        return 0

    df['Angle change'] = df.apply(calculate_angle_change, axis=1)

    # Temps écoulé depuis le dernier événement (en secondes), en prenant en compte le changement de période
    def calculate_time_since_last_event(row):
        if row['Period'] != row['prev_period']:  # Si la période a changé
            return 0  # Recommencer à zéro si la période change
        return row['Game seconds'] - row['prev_game_seconds']
    
    df['time_since_last_event'] = df.apply(calculate_time_since_last_event, axis=1)

    # Distance depuis l'événement précédent
    def calculate_distance_from_last(row):
        if pd.isna(row['prev_x_coord']) or pd.isna(row['prev_y_coord']):
            return 0
        return math.sqrt((row['x_coord'] - row['prev_x_coord']) ** 2 + 
                         (row['y_coord'] - row['prev_y_coord']) ** 2)

    df['distance_from_last_event'] = df.apply(calculate_distance_from_last, axis=1)

    # Vitesse : Distance divisée par le temps écoulé
    def calculate_speed(row):
        if row['time_since_last_event'] > 0:
            return row['distance_from_last_event'] / row['time_since_last_event']
        return 0

    df['Speed'] = df.apply(calculate_speed, axis=1)

    # Retourner le DataFrame avec les nouvelles colonnes ajoutées
    return df

df_ad = add_advanced_features(df)
df_ad.head(80)


,ID,Sort Order,Time,Period,Event ID,Team,Type,Coordinates,Shooter ID,Goalie ID,...,prev_x_coord,prev_y_coord,prev_shot_angle,prev_game_seconds,Rebound,Angle change,time_since_last_event,distance_from_last_event,Speed,prev_period
0,2020020001,10,00:16,1,53,Flyers,shot-on-goal,"(-74, 29)",Travis,Tristan,...,NaN,NaN,NaN,NaN,False,0.000000,0.0,0.000000,0.000000,NaN
1,2020020001,13,00:34,1,9,Penguins,shot-on-goal,"(49, -25)",Evan,Carter,...,-74.0,29.0,117.349876,16.0,True,149.355259,18.0,134.331679,7.462871,1.0
2,2020020001,19,01:05,1,55,Flyers,shot-on-goal,"(-52, -31)",Joel,Tristan,...,49.0,-25.0,-32.005383,34.0,True,108.037068,31.0,101.178061,3.263808,1.0
3,2020020001,39,02:51,1,62,Penguins,shot-on-goal,"(43, 39)",Evan,Carter,...,-52.0,-31.0,-140.042451,65.0,True,180.334599,106.0,118.004237,1.113248,1.0
4,2020020001,46,03:44,1,65,Flyers,shot-on-goal,"(-53, 8)",Claude,Tristan,...,43.0,39.0,40.292148,171.0,True,127.179045,53.0,100.881118,1.903417,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2020020002,150,15:29,1,141,Blackhawks,shot-on-goal,"(-65, 5)",Brandon,Andrei,...,-41.0,-18.0,-159.443955,858.0,True,327.675666,71.0,33.241540,0.468191,1.0
76,2020020002,152,16:05,1,142,Lightning,shot-on-goal,"(75, -14)",Anthony,Malcolm,...,-65.0,5.0,168.231711,929.0,True,213.231711,36.0,141.283403,3.924539,1.0
77,2020020002,154,16:17,1,143,Lightning,shot-on-goal,"(96, 9)",Alex,Malcolm,...,75.0,-14.0,-45.000000,965.0,True,172.874984,12.0,31.144823,2.595402,1.0
78,2020020002,155,16:18,1,144,Lightning,goal,"(80, 0)",Mathieu,Malcolm,...,96.0,9.0,127.874984,977.0,True,127.874984,1.0,18.357560,18.357560,1.0
